In [1]:
import csv
import math
import sys, time
import networkx as nx
import numpy as np
import collections
from pyomo.opt import SolverStatus, TerminationCondition
from pyomo.environ import *

In [2]:
G = nx.Graph()
pos = {}
site = {}

# with open('nodes_cost266.csv', 'r') as f_node:
# with open('nodes_cost_2_commodity.csv', 'r') as f_node:
with open('w_node.csv', 'r') as f_node:
    reader = csv.DictReader(f_node, delimiter=';', quotechar="'")
    for row in reader:
        node = int(row['node'])
        site_node = row['site']
        x_coord = float(row['X'])
        y_coord = float(row['Y'])
        G.add_node(node,site=site_node)
        pos[node] = [x_coord,y_coord]
        site[node] = site_node

In [3]:
# with open('edges_cost266.csv', 'r') as f_edge:
# with open('edges_cost_2_commodity.csv', 'r') as f_edge:
with open('w_edge.csv', 'r') as f_edge:
    reader = csv.DictReader(f_edge, delimiter=';', quotechar="'")
    for row in reader:
        nodeA = int(row['nodeA'])
        nodeB = int(row['nodeB'])
        G.add_edge(nodeA,nodeB)        
        G[nodeA][nodeB]['capa'] = float(row['capa'])

In [4]:
H = nx.DiGraph(G)
capa = {}

for (u,v) in G.edges():
    capa[(u,v)] = G[u][v]['capa']
    capa[(v,u)] = G[u][v]['capa']

In [5]:
K = nx.DiGraph()

# with open('demand_cost266.csv', 'r') as f_demand:
# with open('demand_2_commodity.csv', 'r') as f_demand:
with open('w_demand.csv', 'r') as f_demand:
    reader = csv.DictReader(f_demand, delimiter=';', quotechar="'")
    for row in reader:
        nodeA = int(row['nodeA'])
        nodeB = int(row['nodeB'])
        if nodeA > nodeB:
            tmp = nodeA
            nodeA = nodeB
            nodeB = tmp
            
        K.add_edge(nodeA,nodeB)                    
        # K[nodeA][nodeB]['demand'] = float(row['demand'])/25.0 #Pour arriver à des taux de charges raisonnables
        K[nodeA][nodeB]['demand'] = float(row['demand'])
        # print(float(row['demand']))

In [6]:
print(len(G.nodes()),'nodes')
print(len(G.edges()),'edges')
print(len(K.edges()),'demands')

5 nodes
6 edges
4 demands


In [7]:
import matplotlib.pyplot as plt

%matplotlib inline

plt.figure(figsize=(10,10));
plt.axis('off')

nx.draw_networkx_labels(G,pos,labels=site,font_size=10,font_color='black')

nx.draw(G, pos=pos, node_size=500, node_color='lightgrey', edge_color='grey')

nx.draw_networkx_labels(K,pos,labels=site,font_size=10,font_color='black')  
nx.draw_networkx_nodes(K, pos=pos, node_size=600, node_color='orange',node_shape='s')

In [8]:
# +++ Flow Conservation constraints +++
def flowConservation(model, node, o, d):
    inFlow = sum(
        model.flow[i, j, o, d]
        for (i, j) in model.arcs
        if j == node
    )
    outFlow = sum(
        model.flow[i, j, o, d]
        for (i, j) in model.arcs
        if i == node
    )
    if node == o:
        return (inFlow - outFlow + K[o][d]['demand'] == 0)
    elif node == d:
        return (inFlow - outFlow - K[o][d]['demand'] == 0)
    else:
        return (inFlow == outFlow)


In [9]:
#+++ Capacity constraints +++
def linkCapacity(model, i, j):
    expression = sum(model.flow[i,j,o,d] + model.flow[j,i,o,d] for (o,d) in model.demands)<= model.capa[i,j] * model.gamma
    return expression

In [10]:
solver = 'cplex'

print('>>> Loading optimization model')

model = ConcreteModel()

model.dual = Suffix(direction=Suffix.IMPORT)

model.nodes   = H.nodes()
model.links   = G.edges()
model.arcs    = H.edges()
model.demands = K.edges()

model.index1 = Set(initialize=[(u, o, d) for u in model.nodes for (o, d) in model.demands])
model.index2 = Set(initialize=[(u, v, o, d) for (u, v) in model.arcs for (o, d) in model.demands])

model.capa = Param(model.arcs, initialize=capa)

model.flow  = Var(model.index2, domain=NonNegativeReals)
model.gamma = Var(domain=NonNegativeReals)

model.objective = Objective(rule=lambda m: m.gamma, sense=minimize)

model.flowConserv = Constraint(model.index1, rule=flowConservation)

model.linkCapa = Constraint(model.links, rule=linkCapacity)


>>> Loading optimization model


In [11]:
print('>>> Solving optimization model')
    
# model.preprocess()

optim = SolverFactory(solver)
    
start_time = time.time()

results = optim.solve(model)
    
cpu_time = time.time() - start_time

print('+ TERM STAT:',results.solver.status)
print('+ TERM COND:',results.solver.termination_condition)
    
print('+ CPU  TIME = {0:.2f} s'.format(cpu_time))

if results.solver.status == SolverStatus.ok and results.solver.termination_condition != TerminationCondition.infeasible:
    print('+ OPT VAL  = {0:.8f}'.format(model.gamma.value))
    print('+ LWR BND  = {0:.2f}'.format(results.Problem._list[0]['Lower bound']))

>>> Solving optimization model
+ TERM STAT: ok
+ TERM COND: optimal
+ CPU  TIME = 0.13 s
+ OPT VAL  = 1.33333333
+ LWR BND  = 1.33


In [12]:
epsilon = 1.0e-06

load = {}
    
for (u,v) in G.edges():
    TotalFlow = 0.0
    for (o,d) in K.edges():
        TotalFlow += model.flow[u,v,o,d].value  #On somme pour chaque lien le flow qui passe
        TotalFlow += model.flow[v,u,o,d].value
    G[u][v]['flow'] = TotalFlow    
    if TotalFlow > epsilon:
        load[(u,v)] = 100*TotalFlow/model.capa[(u,v)]
        # print(load[(u,v)])
        print('link ({0:10s},{1:10s}):\tflow = {2:8f} / capacity = {3:8f} -> load = {4:.2f} %'.format(site[u],site[v],TotalFlow,model.capa[(u,v)],load[(u,v)]))
        #print('link ({0:10s},{1:10s}):\tflow = {2:8f} / capacity = {3:8f} -> load = {4:.2f} %'.format(site[u],site[v],TotalFlow,model.capa[(u,v)],100*TotalFlow/model.capa[(u,v)]))
    else:
        load[(u,v)] = 0.0

link (Amsterdam ,Belgrade  ):	flow = 1.333333 / capacity = 1.000000 -> load = 133.33 %
link (Amsterdam ,Berlin    ):	flow = 1.333333 / capacity = 1.000000 -> load = 133.33 %
link (Athens    ,Belgrade  ):	flow = 1.333333 / capacity = 1.000000 -> load = 133.33 %
link (Athens    ,Berlin    ):	flow = 1.333333 / capacity = 1.000000 -> load = 133.33 %
link (Barcelona ,Belgrade  ):	flow = 1.333333 / capacity = 1.000000 -> load = 133.33 %
link (Barcelona ,Berlin    ):	flow = 1.333333 / capacity = 1.000000 -> load = 133.33 %


In [13]:
model.gamma.value
load1 = {}
G1 = G.copy()
Saturated_edge = []

print('____ Saturated Edges _____ ' )
for (u,v) in G1.edges():
    TotalFlow1 = 0.0
    for (o,d) in K.edges():
        TotalFlow1 += model.flow[u,v,o,d].value  #On somme pour chaque lien le flow qui passe
        TotalFlow1 += model.flow[v,u,o,d].value
    G1[u][v]['flow'] = TotalFlow1    
    if abs(100*TotalFlow1/model.capa[(u,v)] - model.gamma.value*100) < epsilon:
        Saturated_edge.append((u,v))
        load1[(u,v)] = 100*TotalFlow1/model.capa[(u,v)]
        print('link ({0:10s}, {1:10s}):\tflow = {2:8f} / capacity = {3:8f} -> load = {4:.2f} %'.format(site[u],site[v],TotalFlow1,model.capa[(u,v)],load1[(u,v)]))
        # model.capa[(u,v)] *= 0.9

# print(Saturated_edge)
def isSaturated(uv):
    isAttained = np.zeros(np.max(G.edges)+1)
    queue = collections.deque()
    u = uv[0]
    v = uv[1]
    isAttained[u] = 1
    for n in G.neighbors(u) : 
        if n not in queue and  abs(load[min(u,n), max(u,n)]  - model.gamma.value*100) > epsilon:
            queue.append(n)
    # for n in G.neighbors(u):
    #     if n not in queue:
    #         queue.append(n)
    while isAttained[v] == 0 and len(queue) != 0:
        n = queue.popleft()
        isAttained[n] = 1
        for n1 in G.neighbors(n) :
            if (n1 not in queue) and (isAttained[n1] == 0) and abs(load[min(n,n1), max(n,n1)]  - model.gamma.value*100) > epsilon:
                queue.append(n1)
    return isAttained[v] == 0 

critical_edge = []
for uv in Saturated_edge : 
    # print(isSaturated(uv))
    if isSaturated(uv):
        critical_edge.append(uv)
print('')
print('_____ Critical Edges ____')
for (u,v) in critical_edge:
    print('link ({0:10s}, {1:10s})\t'.format(site[u],site[v]),'site 1 = '+ str(u)+ ', site 2 = ' + str(v))
print([site[critical_edge[k][0]] + ' -> ' + site[critical_edge[k][1]] for k in range(len(critical_edge))])


____ Saturated Edges _____ 
link (Amsterdam , Belgrade  ):	flow = 1.333333 / capacity = 1.000000 -> load = 133.33 %
link (Amsterdam , Berlin    ):	flow = 1.333333 / capacity = 1.000000 -> load = 133.33 %
link (Athens    , Belgrade  ):	flow = 1.333333 / capacity = 1.000000 -> load = 133.33 %
link (Athens    , Berlin    ):	flow = 1.333333 / capacity = 1.000000 -> load = 133.33 %
link (Barcelona , Belgrade  ):	flow = 1.333333 / capacity = 1.000000 -> load = 133.33 %
link (Barcelona , Berlin    ):	flow = 1.333333 / capacity = 1.000000 -> load = 133.33 %

_____ Critical Edges ____
link (Amsterdam , Belgrade  )	 site 1 = 1, site 2 = 4
link (Amsterdam , Berlin    )	 site 1 = 1, site 2 = 5
link (Athens    , Belgrade  )	 site 1 = 2, site 2 = 4
link (Athens    , Berlin    )	 site 1 = 2, site 2 = 5
link (Barcelona , Belgrade  )	 site 1 = 3, site 2 = 4
link (Barcelona , Berlin    )	 site 1 = 3, site 2 = 5
['Amsterdam -> Belgrade', 'Amsterdam -> Berlin', 'Athens -> Belgrade', 'Athens -> Berlin', 'B

In [14]:
for (o,d) in K.edges():
    print('demand ({0:s},{1:s}): {2:f}'.format(site[o],site[d],K[o][d]['demand']*model.gamma.value))
    # print('demand ({0:s},{1:s}): {2:f}'.format(site[o],site[d],K[o][d]['demand']))
    for (u,v) in model.arcs:
        if model.flow[u,v,o,d].value > epsilon:
            # print('\tarc ({0:s},{1:s}): flow = {2:f}'.format(site[u],site[v],model.flow[u,v,o,d].value/model.gamma.value))
            print('\tarc ({0:s},{1:s}): flow = {2:f}'.format(site[u],site[v],model.flow[u,v,o,d].value))


demand (Belgrade,Berlin): 1.333333
	arc (Amsterdam,Berlin): flow = 0.333333
	arc (Athens,Berlin): flow = 0.333333
	arc (Barcelona,Berlin): flow = 0.333333
	arc (Belgrade,Amsterdam): flow = 0.333333
	arc (Belgrade,Athens): flow = 0.333333
	arc (Belgrade,Barcelona): flow = 0.333333
demand (Amsterdam,Athens): 1.333333
	arc (Amsterdam,Belgrade): flow = 0.500000
	arc (Amsterdam,Berlin): flow = 0.500000
	arc (Belgrade,Athens): flow = 0.500000
	arc (Berlin,Athens): flow = 0.500000
demand (Amsterdam,Barcelona): 1.333333
	arc (Amsterdam,Belgrade): flow = 0.500000
	arc (Amsterdam,Berlin): flow = 0.500000
	arc (Belgrade,Barcelona): flow = 0.500000
	arc (Berlin,Barcelona): flow = 0.500000
demand (Athens,Barcelona): 1.333333
	arc (Athens,Belgrade): flow = 0.500000
	arc (Athens,Berlin): flow = 0.500000
	arc (Belgrade,Barcelona): flow = 0.500000
	arc (Berlin,Barcelona): flow = 0.500000


In [15]:
criticalSources = []
criticalDemands = []

def getCriticalSources():
    criticalSource = []
    criticalDemand = []
    criticalDestination = []
    for (o,d) in K.edges():
        # print('demand ({0:s},{1:s}): {2:f}'.format(site[o],site[d],K[o][d]['demand']*model.gamma.value))
        # print('demand ({0:s},{1:s}): {2:f}'.format(site[o],site[d],K[o][d]['demand']))
        bool = False
        for (u,v) in model.arcs:
            if (min(u,v), max(u,v)) in critical_edge and model.flow[u,v,o,d].value > epsilon:
                # print('\tarc ({0:s},{1:s}): flow = {2:f}'.format(site[u],site[v],model.flow[u,v,o,d].value/model.gamma.value))
                # print('\tarc ({0:s},{1:s}): flow = {2:f}'.format(site[u],site[v],model.flow[u,v,o,d].value))
                bool = True
        if (bool):
            criticalDemand.append((o,d))
            criticalSource.append(o)
            criticalDestination.append(d)
    return criticalDemand, criticalSource, criticalDestination

criticalDemands, criticalSources, criticalDestinations = getCriticalSources()


print('_____ Critical Demands _____')
for (o,d) in criticalDemands : 
    print('demand ({0:s},{1:s}): {2:f}'.format(site[o],site[d],K[o][d]['demand']))

def getSameSideNodes(criticalSources, k):
    sameSideNodes = [criticalSources[k]]
    isAttained = np.zeros(np.max(G.edges) + 1)
    criticalChoosenEdges = []
    queue = collections.deque()
    u = criticalSources[k]
    # print(u)
    isAttained[u] = 1
    for n in G.neighbors(u) : 
        if n not in queue :
            if (min(n,u), max(n,u)) not in critical_edge :
                queue.append(n)
            else :
                if (min(n,u), max(n,u)) not in criticalChoosenEdges:
                        criticalChoosenEdges.append((min(n,u), max(n,u)))
                        # print((min(n,u), max(n,u)) in critical_edge)

    # for n in G.neighbors(u):
    #     if n not in queue:
    #         queue.append(n)
    while len(queue) != 0:
        n = queue.popleft()
        isAttained[n] = 1
        # if n in criticalSources : 
        sameSideNodes.append(n)
        for n1 in G.neighbors(n) :
            if (n1 not in queue) and (isAttained[n1] == 0) :
                if (min(n,n1), max(n,n1)) not in critical_edge :
                    queue.append(n1)
                else :
                    if (min(n,n1), max(n,n1)) not in criticalChoosenEdges:
                        criticalChoosenEdges.append((min(n,n1), max(n,n1)))
                        # print((min(n,n1), max(n,n1)) in critical_edge)
    return sameSideNodes, criticalChoosenEdges


def minCut() : 
    min = np.infty
    minCap = 0
    minDemand = 0
    for k in range(len(criticalSources)):
        sameSideNodes, criticalChoosenEdges = getSameSideNodes(criticalSources, k)
        demand = 0
        for n in sameSideNodes:
            for (o,d) in criticalDemands :
                if (n == o) or n == d:
                    demand += K[o][d]['demand']
        capacity = 0
        
        for (u,v) in criticalChoosenEdges:
            capacity += G[u][v]['capa']
        if (min > capacity/demand):
            min = capacity / demand
            minCap = capacity
            minDemand = demand
    # print(minDemand)
    # print(minCap)
    return min, sameSideNodes, criticalChoosenEdges

mincut, sameSideNodes, criticalChoosenEdges = minCut()

print('')
print('_____ Critical Choosen Edges _____')
for (u,v) in criticalChoosenEdges:
    print('link ({0:10s}, {1:10s})\t'.format(site[u],site[v]),'site 1 = '+ str(u)+ ', site 2 = ' + str(v), ', capa = ', G[u][v]['capa'])
# print(criticalChoosenEdges)


# print(sameSideNodes)
print("")
print('mincut = ',mincut)
print('maxflow = ', 1/model.gamma.value)

print("")
print(sameSideNodes)

_____ Critical Demands _____
demand (Belgrade,Berlin): 1.000000
demand (Amsterdam,Athens): 1.000000
demand (Amsterdam,Barcelona): 1.000000
demand (Athens,Barcelona): 1.000000

_____ Critical Choosen Edges _____
link (Athens    , Belgrade  )	 site 1 = 2, site 2 = 4 , capa =  1.0
link (Athens    , Berlin    )	 site 1 = 2, site 2 = 5 , capa =  1.0

mincut =  1.0
maxflow =  0.75

[2]


In [16]:
plt.figure(figsize=(10,10));
plt.axis('off')

colors = []
for (u,v) in G.edges():
    if (u,v) in criticalChoosenEdges:
        colors.append('blue')
    elif load[(u,v)] > 100:
        colors.append('black')
    elif load[(u,v)] > model.gamma.value *100 - epsilon:
        colors.append('red')
    elif load[(u,v)] > 50:
        colors.append('orange')
    elif load[(u,v)] >=25:
        colors.append('green')
    elif load[(u,v)] >=0:
        colors.append('lightgreen')
    else:
        colors.append('lightgrey')

    nx.draw_networkx_labels(G,pos,labels=site,font_size=10,font_color='black')
    nx.draw(G, pos=pos, node_size=500, node_color='lightgrey', edge_color=colors, width=4)

    nx.draw_networkx_labels(K,pos,labels=site,font_size=10,font_color='black')  
    nx.draw_networkx_nodes(K, pos=pos, node_size=600, node_color='orange',node_shape='s')